# Computing ALFF
Imput data should not be band pass and standardize 

## <font color=#B2D732> <span style="background-color: #4424D6"> Imports

In [9]:
import sys, glob
main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/"

sys.path.append(main_dir + "/2025_brsc_aging_project/code/")
from connectivity.alff import ALFF
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nibabel as nib
import os.path
import json

from plotting import Plotting

# Set numpy to print only 2 decimal digits for neatness
np.set_printoptions(precision=2, suppress=True)

# bibliography:
# https://doi.org/10.1016/j.neuroimage.2009.09.037

%matplotlib inline
%load_ext autoreload
%autoreload 2
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=#B2D732> <span style="background-color: #4424D6"> Load configuration

In [24]:
with open('../../config/analyses/brsc_functional.json') as config_file:
    config = json.load(config_file)

## <font color=#B2D732> <span style="background-color: #4424D6"> 1. ALFF computations

#### A. Create instance of ALFF class for further analyses

In [25]:
#redo with smoothed data to compare
structure='spinalcord'
ana="alff"
alff = ALFF(config,IDs=config["participants_IDs_ALL"],structure=structure,analysis=ana) 

Output directory: /cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project//figures/f02_functional_sc/alff_inPAM50_nostd_oct25/spinalcord//alff/slow5/


#### B. Compute individual maps

In [26]:
fwhm=[3,3,3] if structure =="brain" else [1,1,1]
alff_files,zalff_files=alff.compute_alff_maps(fwhm=fwhm,scaling_method="zscore",redo=False,n_jobs=4)

ALFF MAPS COMPUTATION
Overwrite old files: False
... Preparing data
... alff is Running
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Batch computation too fast (0.03448152542114258s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Batch computation too fast (0.013869285583496094s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  14 tasks

## <font color=#B2D732> <span style="background-color: #4424D6"> 2. ALFF ROI analyses

### A. Compute values per ROI
- within gm mask

In [30]:
alff_df={}

if structure =="brain":
    atlas={"schaefer200_cobra":"schaefer200_cobra"}
    template_dir=config["project_dir"] + "/templates/MNI/atlas/"
    tag_sup="/"
else:
    atlas={"levels":"_"}
    template_dir=config["project_dir"] + "/templates/PAM50/atlas/Frostell"
    tag_sup="28Parcels_7Networks/Frostell"

#---- Load image and label file
for tag_nb, tag in enumerate(atlas):
    atlas_f=glob.glob(template_dir+atlas[tag]+tag_sup+atlas[tag]+"*.nii.gz")[0]
    atlas_labels=glob.glob(template_dir+atlas[tag]+tag_sup+atlas[tag]+"*.txt")[0]
    labelslist=np.genfromtxt(atlas_labels, usecols=1,skip_header=1, dtype="S", delimiter="\t", encoding=None).astype(str)
    labelsnb=np.genfromtxt(atlas_labels, usecols=0,skip_header=1, dtype="S", delimiter="\t", encoding=None).astype(str)
    alff_df[tag]=alff.extract_alff_rois(input_f=alff_files,atlas_f=atlas_f,atlas_labels=labelslist,atlas_labels_nb=labelsnb,tag="_" + tag,redo=False,plot=False,verbose=0)



ALFF IN ROIS


In [29]:
atlas_f

'/cerebro/cerebro1/dataset//templates/PAM50/template/PAM50_gm_01bin_C1C7.nii.gz'